# memstack-agent 工具系统基础

本 notebook 演示如何使用 memstack-agent 框架创建和管理工具。

## 概述

memstack-agent 使用 Protocol-based 的工具接口，支持：
- 函数装饰器方式创建工具
- 类继承方式创建工具
- 自动 JSON Schema 推断
- OpenAI 和 Anthropic 格式转换

## 1. 导入必要模块

In [ ]:
import sys
sys.path.insert(0, '/Users/tiejunsun/github/agi-demos/src')

from memstack_agent import (
    function_to_tool,
    infer_type_schema,
    ToolDefinition,
    Tool,
    ToolMetadata,
)
from memstack_agent.tools.protocol import SimpleTool

## 2. 使用 function_to_tool 装饰器

最简单的创建工具方式是使用 `function_to_tool` 装饰器。
框架会自动从函数签名和类型提示推断 JSON Schema。

In [ ]:
@function_to_tool
async def search_web(query: str, max_results: int = 5) -> str:
    """搜索互联网获取信息。
    
    Args:
        query: 搜索查询词
        max_results: 返回结果数量，默认 5
    """
    # 模拟搜索结果
    return f"Found {max_results} results for '{query}'"


# 查看生成的工具定义
print(f"Tool name: {search_web.name}")
print(f"Description: {search_web.description}")
print(f"Parameters schema: {search_web.parameters}")

In [ ]:
# 执行工具
import asyncio

result = await search_web.execute(query="Python async", max_results=3)
print(f"Result: {result}")

## 3. 查看生成的 JSON Schema

框架自动从 Python 类型提示推断 JSON Schema。

In [ ]:
import json

print("Tool JSON Schema:")
print(json.dumps(search_web.parameters, indent=2, ensure_ascii=False))

## 4. 类型推断示例

使用 `infer_type_schema` 函数可以直接推断类型的 JSON Schema。

In [ ]:
from typing import Optional, List, Dict, Union

# 基础类型
print("str:", infer_type_schema(str))
print("int:", infer_type_schema(int))
print("float:", infer_type_schema(float))
print("bool:", infer_type_schema(bool))

# Optional 类型
print("\nOptional[str]:", infer_type_schema(Optional[str]))

# List 类型
print("List[str]:", infer_type_schema(List[str]))
print("List[int]:", infer_type_schema(List[int]))

# Dict 类型
print("\nDict[str, str]:", infer_type_schema(Dict[str, str]))
print("Dict[str, int]:", infer_type_schema(Dict[str, int]))

# Union 类型
print("\nUnion[str, int]:", infer_type_schema(Union[str, int]))

## 5. 转换为 LLM 格式

`ToolDefinition` 支持直接转换为 OpenAI 和 Anthropic 的工具格式。

In [ ]:
# OpenAI 格式
print("OpenAI format:")
print(json.dumps(search_web.to_openai_format(), indent=2, ensure_ascii=False))

In [ ]:
# Anthropic 格式
print("Anthropic format:")
print(json.dumps(search_web.to_anthropic_format(), indent=2, ensure_ascii=False))

## 6. 使用 SimpleTool 类

对于更复杂的工具，可以继承 `SimpleTool` 类。

In [ ]:
class CalculatorTool(SimpleTool):
    """计算器工具示例。"""
    
    name = "calculator"
    description = "执行数学计算，支持加减乘除"
    
    async def execute(self, expression: str) -> str:
        """执行数学表达式。
        
        Args:
            expression: 数学表达式，如 '2 + 3 * 4'
        
        Returns:
            计算结果
        """
        try:
            # 注意：实际生产中应使用更安全的表达式解析
            result = eval(expression)
            return str(result)
        except Exception as e:
            return f"Error: {e}"
    
    def get_parameters_schema(self) -> dict:
        return {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "数学表达式，如 '2 + 3 * 4'"
                }
            },
            "required": ["expression"]
        }


# 创建工具实例
calc = CalculatorTool()

# 使用 function_to_tool 包装
calc_def = function_to_tool(
    calc.execute,
    name=calc.name,
    description=calc.description,
)

print(f"Tool: {calc_def.name}")
print(f"Description: {calc_def.description}")

# 执行
result = await calc_def.execute(expression="2 + 3 * 4")
print(f"2 + 3 * 4 = {result}")

## 7. 工具元数据

使用 `ToolMetadata` 添加额外的工具信息。

In [ ]:
# 创建带元数据的工具
metadata = ToolMetadata(
    tags=["web", "search", "information"],
    visible_to_model=True,
    timeout_seconds=30,
    ui_category="Web Tools",
    extra={"rate_limit": "100/hour"}
)

@function_to_tool
async def fetch_url(url: str) -> str:
    """获取网页内容。
    
    Args:
        url: 要获取的网页 URL
    """
    return f"Content from {url}"

# 注意：function_to_tool 装饰器会创建新的 ToolDefinition
# 元数据可以通过 metadata 参数传入
fetch_tool = function_to_tool(
    fetch_url.execute,
    name="fetch_url",
    description="获取网页内容",
    metadata=metadata,
)

print("Tool metadata:")
print(json.dumps(fetch_tool.to_dict(), indent=2, ensure_ascii=False))

## 8. 工具列表管理

在实际应用中，通常需要管理多个工具。

In [ ]:
# 创建多个工具
@function_to_tool
async def read_file(path: str) -> str:
    """读取文件内容。
    
    Args:
        path: 文件路径
    """
    return f"Content of {path}"


@function_to_tool
async def write_file(path: str, content: str) -> str:
    """写入文件。
    
    Args:
        path: 文件路径
        content: 要写入的内容
    """
    return f"Wrote {len(content)} chars to {path}"


@function_to_tool
async def list_files(directory: str = ".") -> str:
    """列出目录中的文件。
    
    Args:
        directory: 目录路径，默认当前目录
    """
    return f"Files in {directory}"


# 工具集合
tools = [read_file, write_file, list_files]

print("Available tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description[:50]}...")

In [ ]:
# 转换为 OpenAI 工具列表格式
openai_tools = [tool.to_openai_format() for tool in tools]

print("OpenAI tools format:")
print(json.dumps(openai_tools, indent=2, ensure_ascii=False))

## 总结

本 notebook 演示了 memstack-agent 工具系统的核心功能：

1. **function_to_tool 装饰器** - 快速将函数转换为工具
2. **自动类型推断** - 从 Python 类型提示生成 JSON Schema
3. **格式转换** - 支持 OpenAI 和 Anthropic 格式
4. **SimpleTool 类** - 用于更复杂的工具实现
5. **ToolMetadata** - 添加额外元数据

下一步：查看 `02-events-and-state.ipynb` 了解事件系统。